In [3]:
from enum import Enum

class Color(Enum):
    WHITE = 0
    BLACK = 1
    
def algebraic_to_index(algebraic):
        return ord(algebraic[0].upper()) - 65, int(algebraic[1])-1

def index_to_algebraic(index):
    return chr(index[0]+65) + str(index[1]+1)
    
def pos_within_board(x, y):
    return x >= 0 and x < 8 and y >= 0 and y < 8

straight_moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]
diagonal_moves = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
knight_moves = [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (-1, 2), (1, -2), (-1, -2)]

In [12]:
class piece:
    def __init__(self, color):
        self.color = color
        self.hasBeenMoved = False
        
    def move_piece(self, game, move):
        target_x, target_y = algebraic_to_index(move[-2:])
        current_x, current_y = algebraic_to_index(move[:2])
        if not (pos_within_board(target_x, target_y) and pos_within_board(current_x, current_y)):
            print('Out of bounds')
            return False
        available_moves = self.possible_moves(game, current_x, current_y)
        if (target_x, target_y) in available_moves:
            attacked_piece = game.board[target_y][target_x]
            en_passant = type(self) is pawn and target_x != current_x and attacked_piece is None
            if en_passant: 
                attacked_piece = game.board[current_y][target_x]
                game.board[current_y][target_x] = None
            game.board[target_y][target_x] = self
            game.board[current_y][current_x] = None
            if game.is_king_checked(self.color):
                print('The king is in check. Try a different move.')
                #returns the board to the previous state
                if en_passant:
                    game.board[current_y][target_x] = attacked_piece
                else:
                    game.board[target_y][target_x] = attacked_piece
                game.board[current_y][current_x] = self
                return False
            #Check if promotable pawn
            if type(self) is pawn and (target_y == 7 or target_y == 0): self.__class__ = queen
            #Check for possible en passant move next round
            if type(self) is pawn and abs(target_y - current_y) == 2:
                if pos_within_board(target_x-1, target_y) and type(game.board[target_y][target_x-1]) is pawn and game.board[target_y][target_x-1].color != self.color:
                    game.board[target_y][target_x-1].en_passant = {'round' : game._total_moves+1, 'column' : target_x }
                if pos_within_board(target_x+1, target_y) and type(game.board[target_y][target_x+1]) is pawn and game.board[target_y][target_x+1].color != self.color:
                    game.board[target_y][target_x+1].en_passant = {'round' : game._total_moves+1, 'column' : target_x }
            self.hasBeenMoved = True
            return True
        else:
            print('Not possible to perform this move')
            print('Available moves for {} at {}: {}'.format(self, index_to_algebraic((current_x, current_y)), [index_to_algebraic(move) for move in available_moves]))
            return False
        
class pawn(piece):
    def __str__(self):
        if self.color == Color.WHITE:
            return '♙'
        else:
            return '♟'
    
    def __init__(self, color):
        super().__init__(color)
        self.en_passant = None
    
    def possible_moves(self, game, pos_x, pos_y):
        valid_moves = []
        valid_y_direction = -1
        if self.color == Color.WHITE: valid_y_direction = 1
        #One step ahead
        if pos_within_board(pos_y+valid_y_direction, pos_x) and game.board[pos_y+valid_y_direction][pos_x] is None:
            valid_moves.append((pos_x, pos_y+valid_y_direction))
            #Two steps ahead
            if not self.hasBeenMoved and pos_within_board(pos_y + 2*valid_y_direction, pos_x) and game.board[pos_y + 2*valid_y_direction][pos_x] is None:
                valid_moves.append((pos_x, pos_y + 2*valid_y_direction))
        #Attack left flank
        if pos_within_board(pos_y+valid_y_direction, pos_x - 1) and game.board[pos_y+valid_y_direction][pos_x - 1] is not None:
            if game.board[pos_y+valid_y_direction][pos_x -1].color is not self.color:
                valid_moves.append((pos_x - 1, pos_y+valid_y_direction))
        #Attack right flank
        if pos_within_board(pos_y+valid_y_direction, pos_x + 1) and game.board[pos_y+valid_y_direction][pos_x + 1] is not None:
            if game.board[pos_y+valid_y_direction][pos_x + 1].color is not self.color:
                valid_moves.append((pos_x + 1, pos_y+valid_y_direction))
        #En passant
        if not self.en_passant is None:
            if game._total_moves == self.en_passant['round']:
                valid_moves.append((self.en_passant['column'], pos_y+valid_y_direction))
        return valid_moves
        
class rook(piece):
    def __str__(self):
        if self.color == Color.WHITE:
            return '♖'
        else:
            return '♜'
        
    def possible_moves(self, game, pos_x, pos_y):
        valid_moves = []
        for move in straight_moves:
            squares = 1
            test_x, test_y = tuple(map(sum, zip((pos_x, pos_y),move*squares )))
            while pos_within_board(test_x, test_y):
                if game.board[test_y][test_x] is None:
                    valid_moves.append((test_x, test_y))
                elif game.board[test_y][test_x].color is not self.color:
                    valid_moves.append((test_x, test_y))
                    break
                else:
                    break
                squares += 1
                test_x = pos_x + move[0]*squares
                test_y = pos_y + move[1]*squares
        return valid_moves

class knight(piece):
    def __str__(self):
        if self.color == Color.WHITE:
            return '♘'
        else:
            return '♞'
    
    def possible_moves(self, game, pos_x, pos_y):
        valid_moves = []
        for move in knight_moves:
            test_x, test_y = tuple(map(sum, zip((pos_x, pos_y),move )))
            if pos_within_board(test_x, test_y):
                if game.board[test_y][test_x] is None:
                    valid_moves.append((test_x, test_y))
                elif game.board[test_y][test_x].color is not self.color:
                    valid_moves.append((test_x, test_y))
        return valid_moves
    
class bishop(piece):
    def __str__(self):
        if self.color == Color.WHITE:
            return '♗'
        else:
            return '♝'
        
    def possible_moves(self, game, pos_x, pos_y):
        valid_moves = []
        for move in diagonal_moves:
            squares = 1
            test_x, test_y = tuple(map(sum, zip((pos_x, pos_y),move*squares )))
            while pos_within_board(test_x, test_y):
                if game.board[test_y][test_x] is None:
                    valid_moves.append((test_x, test_y))
                elif game.board[test_y][test_x].color is not self.color:
                    valid_moves.append((test_x, test_y))
                    break
                else:
                    break
                squares += 1
                test_x = pos_x + move[0]*squares
                test_y = pos_y + move[1]*squares
        return valid_moves
        
class king(piece):
    def __str__(self):
        if self.color == Color.WHITE:
            return '♔'
        else:
            return '♚'
        
    def possible_moves(self, game, pos_x, pos_y):
        valid_moves = []
        for movetype in [diagonal_moves, straight_moves]:
            for move in movetype:
                test_x, test_y = tuple(map(sum, zip((pos_x, pos_y),move )))
                if pos_within_board(test_x, test_y):
                    if game.board[test_y][test_x] is None:
                        valid_moves.append((test_x, test_y))
                    elif game.board[test_y][test_x].color is not self.color:
                        valid_moves.append((test_x, test_y))
        return valid_moves
        
class queen(piece):
    def __str__(self):
        if self.color == Color.WHITE:
            return '♕'
        else:
            return '♛'
        
    def possible_moves(self, game, pos_x, pos_y):
        valid_moves = []
        for movetype in [diagonal_moves, straight_moves]:
            for move in movetype:
                squares = 1
                test_x, test_y = tuple(map(sum, zip((pos_x, pos_y),move*squares )))
                while pos_within_board(test_x, test_y):
                    if game.board[test_y][test_x] is None:
                        valid_moves.append((test_x, test_y))
                    elif game.board[test_y][test_x].color is not self.color:
                        valid_moves.append((test_x, test_y))
                        break
                    else:
                        break
                    squares += 1
                    test_x = pos_x + move[0]*squares
                    test_y = pos_y + move[1]*squares
        return valid_moves
        
class game:    
        
    def setupBoard(self):
        w = Color.WHITE
        b = Color.BLACK
        self.board = [
            [rook(w), knight(w), bishop(w), queen(w), king(w), bishop(w), knight(w), rook(w)],
            [pawn(w), pawn(w), pawn(w), pawn(w), pawn(w), pawn(w), pawn(w), pawn(w)],
            [None]*8,
            [None]*8,
            [None]*8,
            [None]*8,
            [pawn(b), pawn(b), pawn(b), pawn(b), pawn(b), pawn(b), pawn(b), pawn(b)],
            [rook(b), knight(b), bishop(b), queen(b), king(b), bishop(b), knight(b), rook(b)]]
        
    def __str__(self):
        #Different unicode spaces have been used to attempt to align the pieces and headers.
        boardString = '\u2007 '
        for c in self._A_H:
            boardString += c + '\u200A'
        boardString += '\n'
        for i in range(1,9):
            y = str(9-i)
            boardString += y + ' '
            for x in range(8):
                piece = self.board[8-i][x]
                if(piece == None): boardString += '\u3000'
                else: boardString += str(piece)
            boardString += '\n'
        return boardString
    
    def __init__(self):
        self._A_H = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
        self.board = []
        self.setupBoard()
        self._whiteToMove = True
        self._total_moves = 0
        self.game_over = False
        
    def is_king_checked(self, color):
        #Loops through pieces of opposite color and checks if the king position is in the list of possible moves
        king_position = self._find_king_position(color)
        for row in range(8):
            for col in range(8):
                if self.board[row][col] != None and self.board[row][col].color != color and king_position in self.board[row][col].possible_moves(self, col, row):
                    return True
        return False            
                    
    def _find_king_position(self, color):
        for row in range(8):
            for col in range(8):
                if isinstance(self.board[row][col], king) and self.board[row][col].color == color:
                    return (col, row)
    
    def is_checkmate(self, color):
        king_position = self._find_king_position(color)
        unchecking_moves = []
        for row in range(8):
            for col in range(8):
                #Find pieces of same color
                if self.board[row][col] != None and self.board[row][col].color == color: 
                    #Test if any of the avilable moves can resolve the checkmate
                    for move in self.board[row][col].possible_moves(self, col, row):
                        temp = self.board[move[1]][move[0]]
                        self.board[move[1]][move[0]] = self.board[row][col]
                        self.board[row][col] = None
                        if not self.is_king_checked(color):
                            unchecking_moves.append(move)
                        #Return the pieces to their positions
                        self.board[row][col] = self.board[move[1]][move[0]]
                        self.board[move[1]][move[0]] = temp
        if not unchecking_moves:
            return True
        return False
    
    def _valid_position_format(self, position):
        if(not len(position) == 2):
            return False
        position = position.upper()
        try:
            i = int(position[1])
        except:
            return False
        return (position[0] in self._A_H and i > 0 and i < 9)
    
    def _castle(self, kingside):
        color = Color.BLACK
        if self._whiteToMove: color = Color.WHITE
        king_position = self._find_king_position(color)
        king = self.board[king_position[1]][king_position[0]]
        if king.hasBeenMoved:
            print('Cannot castle. The king has been moved.')
            return
        if kingside:
            if self.board[king_position[1]][7].hasBeenMoved:
                print('Cannot castle. The rook has been moved.')
                return
            if self.board[king_position[1]][6] != None or self.board[king_position[1]][5] != None:
                print('Cannot castle. There are pieces standing in the way.')
                return
            if self.is_king_checked(color):
                print('Cannot castle. The king is checked.')
                return
            self.board[king_position[1]][5] = king
            self.board[king_position[1]][king_position[0]] = None
            if self.is_king_checked(color):
                print('Cannot castle. The kings passage is checked.')
                self.board[king_position[1]][5] = None
                self.board[king_position[1]][king_position[0]] = king
                return
            self.board[king_position[1]][6] = king
            self.board[king_position[1]][5] = None
            if self.is_king_checked(color):
                print('Cannot castle. The king would go into check.')
                self.board[king_position[1]][6] = None
                self.board[king_position[1]][king_position[0]] = king
                return
            #Moving rook
            self.board[king_position[1]][5] = self.board[king_position[1]][7] 
            self.board[king_position[1]][7] = None
            king.hasBeenMoved = True
            self.board[king_position[1]][5].hasBeenMoved = True
            self._total_moves += 1
            self._whiteToMove = not self._whiteToMove
        else:
            if self.board[king_position[1]][0].hasBeenMoved:
                print('Cannot castle. The rook has been moved.')
                return
            if self.board[king_position[1]][1] != None or self.board[king_position[1]][2] != None or self.board[king_position[1]][3] != None:
                print('Cannot castle. There are pieces standing in the way.')
                return
            if self.is_king_checked(color):
                print('Cannot castle. The king is checked.')
                return
            self.board[king_position[1]][3] = king
            self.board[king_position[1]][king_position[0]] = None
            if self.is_king_checked(color):
                print('Cannot castle. The kings passage is checked.')
                self.board[king_position[1]][3] = None
                self.board[king_position[1]][king_position[0]] = king
                return
            self.board[king_position[1]][2] = king
            self.board[king_position[1]][3] = None
            if self.is_king_checked(color):
                print('Cannot castle. The king would go into check.')
                self.board[king_position[1]][2] = None
                self.board[king_position[1]][king_position[0]] = king
                return
            #Moving rook
            self.board[king_position[1]][3] = self.board[king_position[1]][0] 
            self.board[king_position[1]][0] = None
            king.hasBeenMoved = True
            self.board[king_position[1]][3].hasBeenMoved = True
            self._total_moves += 1
            self._whiteToMove = not self._whiteToMove
            
            
    def move(self, move):
        move = move.upper()
        if move == '0-0':
            self._castle(kingside = True)
            return
        elif move == '0-0-0':
            self._castle(kingside = False)
            return

        current_position = move[:2]
        target_position = move[-2:]
        if not self._valid_position_format(current_position):
            print ("'{}' is not a valid position".format(current_position))
            return
        if not self._valid_position_format(target_position):
            print ("'{}' is not a valid position".format(target_position))
            return
        currentX, currentY = algebraic_to_index(current_position)
        pieceToMove = self.board[currentY][currentX]
        if pieceToMove == None:
            print ("No piece in position'{}'".format(current_position))
            return
        if not((pieceToMove.color == Color.WHITE and self._whiteToMove) or (pieceToMove.color == Color.BLACK and not self._whiteToMove)):
            color = 'black'
            if(self._whiteToMove): color = 'white'
            print ('Wrong color. It is {}s turn to move'.format(color))
            return
        
        if pieceToMove.move_piece(self, move):
            self._total_moves += 1
            self._whiteToMove = not self._whiteToMove
            other_color = Color.BLACK
            if pieceToMove.color == Color.BLACK: other_color = Color.WHITE
            if self.is_king_checked(other_color):
                if self.is_checkmate(other_color):
                    self.game_over = True
                else:
                    print('Check!')

from IPython.display import clear_output                    
class chess_client:
    def __init__(self):
        self.game = game()
        self.help = ("The general rule to make a move is to enter 'piece position' 'target positon',"
                     "as eg. 'D2 D4' to move pawn on D2 to D4.\n"
                     "There are some exceptions:\n"
                     "0-0   : Castle kingside\n"
                     "0-0-0 : Castle queenside\n"
                     "exit : Exit the game\n"
                    )
        
    def play(self):
        #print(("Welcome to pychess. To make a move, enter e.g. 'D2 D4' to move the piece in D2 to D4.\n"
         #   "Type 'help' for help. Type 'exit' to exit."))
        cont = True
        while cont:
            print(self.game)
            command = input()
            clear_output()
            print(command)
            command = command.lower()
            if command == 'exit':
                cont = False
            elif command == 'help':
                print(self.help)
            else:
                self.game.move(command)
            if self.game.game_over:
                cont = False
                print('Checkmate!')
                


In [15]:
client = chess_client()
client.play()

exit
